![](../assets/architecture.png)

In this notebook we will make the residual layer with a couple of design decisions since the paper didn't specify
+ There will be two dialted convolution one for each non linearity
+ There will be two 1x1 convolutions one for skip connections and the other for residual connections
+ Clip the inputs when adding to the residual outputs

In [1]:
import torch

In [37]:
residual_channels = 32
skip_channels = 512
dilation = 2

In [3]:
inputs = torch.arange(1, 21, dtype=torch.float).view(1,1,-1)
inputs, inputs.shape

(tensor([[[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
           15., 16., 17., 18., 19., 20.]]]),
 torch.Size([1, 1, 20]))

In [5]:
from .model import DialatedConv1d

ModuleNotFoundError: No module named 'WaveNet'

In [38]:
dialted_conv = DialatedConv1d(1, residual_channels, dilation=dilation)
dialted_conv(inputs).shape

torch.Size([1, 32, 16])

In [39]:
dialted_conv_filter = DialatedConv1d(1, residual_channels, dilation=dilation)
dialted_conv_gate = DialatedConv1d(1, residual_channels, dilation=dilation)

In [9]:
filter_out, gate_out = dialted_conv_filter(inputs), dialted_conv_gate(inputs)
filter_out.shape, gate_out.shape

(torch.Size([1, 32, 16]), torch.Size([1, 32, 16]))

In [11]:
z = torch.tanh(filter_out) * torch.sigmoid(gate_out)
z.shape

torch.Size([1, 32, 16])

In [17]:
import torch.nn as nn

class Conv1d1x1(nn.Conv1d):
    def __init__(self, in_channels, out_channels,
                 bias=False, device=None, dtype=None):
        super(Conv1d1x1, self).__init__(in_channels, out_channels, kernel_size=1,
                                        bias=bias, device=device, dtype=dtype)
    def forward(self, inputs):
        return super(Conv1d1x1, self).forward(inputs)

In [20]:
Conv1d1x1(residual_channels, 100)(z).shape

torch.Size([1, 100, 16])

In [21]:
residual_1x1 = Conv1d1x1(residual_channels, residual_channels)
skip_1x1 = Conv1d1x1(residual_channels, skip_channels)

In [23]:
residual_out, skip_out = residual_1x1(z), skip_1x1(z)
residual_out.shape, skip_out.shape

(torch.Size([1, 32, 16]), torch.Size([1, 512, 16]))

In [24]:
inputs.shape

torch.Size([1, 1, 20])

Since input shape doesn't match the residual output we can clip them

In [25]:
residual_out.size(2)

16

In [31]:
clipped_inputs = inputs[:,:, -residual_out.size(2):]
clipped_inputs, clipped_inputs.shape

(tensor([[[ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18.,
           19., 20.]]]),
 torch.Size([1, 1, 16]))

In [32]:
residual_out += clipped_inputs

In [34]:
residual_out.shape, skip_out.shape

(torch.Size([1, 32, 16]), torch.Size([1, 512, 16]))

This is the output of the residual layer

In [53]:
import torch.nn as nn
from model import DialatedConv1d

class ResidualLayer(nn.Module):
    def __init__(self, dilation, residual_channels=32, skip_channels=512,
                 filter_in_channels=1, gate_in_channels=1):
        super(ResidualLayer, self).__init__()
        self.filter_dialated_conv = DialatedConv1d(filter_in_channels, residual_channels, dilation)
        self.gate_dialted_conv = DialatedConv1d(gate_in_channels, residual_channels, dilation)
        self.residual_1x1 = Conv1d1x1(residual_channels, residual_channels)
        self.skip_1x1 = Conv1d1x1(residual_channels, skip_channels)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, inputs):
        assert inputs.dim() == 3 # To clip the inputs
        filter_out, gate_out = self.filter_dialated_conv(inputs), self.gate_dialted_conv(inputs)
        z = self.tanh(filter_out) * self.sigmoid(gate_out)
        residual_out, skip_out  = self.residual_1x1(z), self.skip_1x1(z)
        clipped_inputs = inputs[:,:, -residual_out.size(2):]
        residual_out += clipped_inputs
        
        return residual_out, skip_out

In [54]:
inputs.dim()

3

In [56]:
res_out, skip_out = ResidualLayer(dilation=2)(inputs)
res_out.shape, skip_out.shape

(torch.Size([1, 32, 16]), torch.Size([1, 512, 16]))